<h2 align="center"> Лабораторна робота №2</h2>
<p align="center">ФБ-23 Ткачук Андрій</p>

<h1> Хід виконання</h1>

##### 1. Створити env в якому будуть встановлені всі необхідні бібліотеки та налаштування для данної лабораторної роботи 

Створив env за допомогою anaconda

##### 2. Для кожної із адміністративних одиниць України завантажити тестові структуровані файли, що містять значення VHI-індексу. Ця процедура має бути автоматизована, параметром процедури має бути індекс (номер) області. При зберіганні файлу до його імені потрібно додати дату та час завантаження;

In [105]:
import datetime
import os
import urllib.request

CurrentTime=datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
dir_path=os.path.join("DataCSV",f"NOAA_{CurrentTime}")
os.makedirs(dir_path)


for provid in range(1,28):
    url=url=f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?provinceID={provid}&country=UKR&yearlyTag=Weekly&type=Mean&TagCropland=crop&year1=1982&year2=2024'
    FileName=f"NOAA_ID{provid}_{CurrentTime}.csv"
    FilePath=os.path.join(dir_path,FileName)

    try:
        urllib.request.urlretrieve(url,FilePath)  
    except Exception as e: print(e)

print(f"Файли встановлено в {dir_path}")

Файли встановлено в DataCSV\NOAA_2024-03-01_20-46-14


##### 3. Зчитати завантажені текстові файли у фрейм (детальніше про роботу із фреймами буде розказано у подальших лабораторних роботах). Імена стовбців фрейму мають бути змістовними та легкими для сприйняття (не повинно бути спеціалізованих символів, пробілів тощо). Ця задача має бути реалізована у вигляді окремої процедури, яка на вхід приймає шлях до директорії, в якій зберігаються файли;

In [113]:
import pandas as pd

def cleandf(MainDir):
    if not os.path.exists(MainDir):
        print(f'Директорія {MainDir} не створена')
    else:

        ls =[]

        dirs=[]
        for file in os.listdir(MainDir): 
            if os.path.isdir(os.path.join(MainDir,file)): 
                dirs.append(file)

        ###Choosing Newest directory
        if len(dirs) == 1:
            NewestDir=os.path.join(MainDir,dirs[0])
        else:
            for i in range(1,len(dirs)):
                if os.path.getctime(os.path.join(MainDir,dirs[i-1])) < os.path.getctime(os.path.join(MainDir,dirs[i])):
                    NewestDir=os.path.join(MainDir,dirs[i])
        #print(NewestDir)

        ##Creating df
        files=os.listdir(NewestDir)
        for i,file in enumerate(files):
            FilePath=os.path.join(NewestDir,file)
            df=pd.read_csv(FilePath,index_col=False,header=1)
            df["ID"]=i+1
            ls.append(df)

        df=pd.concat(ls)

    ###Rename columns
        print(f"Перед редагуванням: {df.columns.values}")
        df=df.rename(columns={' SMN': 'SMN'})
        df=df.rename(columns={' VHI<br>': 'VHI'})
        print(f"Після редагування: {df.columns.values}")
        
    ###Cleaning Nan and -1
        print(f"\n Перед видаленням Nan: \n{df.shape}")
        df=df.dropna()
        df=df.drop(df.loc[df['VHI'] == -1].index)
        print(f"\n Після видалення Nan: \n{df.shape}")

    ###Cleaning html tags
        print(f"\nПеред редагуванням: \n{df['year'].str.len().value_counts()}")
        df['year'].replace({"<tt><pre>1982" : "1982"}, inplace=True)
        print(f"\nПісля редагування:\n{df['year'].str.len().value_counts()}\n")

        return(df)

cdf=cleandf('DataCSV')
print(cdf)
#print(cdf.shape)

Перед редагуванням: ['year' 'week' ' SMN' 'SMT' 'VCI' 'TCI' ' VHI<br>' 'ID']
Після редагування: ['year' 'week' 'SMN' 'SMT' 'VCI' 'TCI' 'VHI' 'ID']

 Перед видаленням Nan: 
(60399, 8)

 Після видалення Nan: 
(57834, 8)

Перед редагуванням: 
year
4     57807
13       27
Name: count, dtype: int64

Після редагування:
year
4    57834
Name: count, dtype: int64

      year  week    SMN     SMT    VCI    TCI    VHI  ID
0     1982   1.0  0.058  258.03  51.18  49.16  50.17   1
1     1982   2.0  0.062  261.39  55.47  38.40  46.94   1
2     1982   3.0  0.061  263.35  56.51  32.77  44.64   1
3     1982   4.0  0.058  264.99  52.86  28.74  40.80   1
4     1982   5.0  0.055  266.32  45.90  28.59  37.25   1
...    ...   ...    ...     ...    ...    ...    ...  ..
2187  2024   4.0  0.093  273.64  54.61  11.58  33.09  27
2188  2024   5.0  0.097  274.40  56.21  11.30  33.76  27
2189  2024   6.0  0.104  275.24  57.40  13.57  35.48  27
2190  2024   7.0  0.111  276.07  55.85  17.90  36.87  27
2191  2024   8.

##### 4. Реалізувати процедуру, яка змінить індекси областей, які використані на порталі NOAA.

In [109]:
###Змінюю індекси (25 з методички + Київ 26 та Севастополь 27)
new_index={
    1:22,
    2:24,
    3:23,
    4:25,
    5:3,
    6:4,   
    7:8,
    8:19,   
    9:20,
    10:21,
    11:9,
    12:26,
    13:10,
    14:11,
    15:12,
    16:13,
    17:14,
    18:15,
    19:16,
    20:27,
    21:17,
    22:18,
    23:6,
    24:1,
    25:2,
    26:7,
    27:5
}

print(f"\t Початкові дані:\n {cdf.head()}")
cdf=cdf.replace({"ID" : new_index})
print(f"\t Дані після зміни індексів: \n {cdf.head()}")

	 Початкові дані:
    year  week    SMN     SMT    VCI    TCI    VHI  ID
0  1982   1.0  0.058  258.03  51.18  49.16  50.17   1
1  1982   2.0  0.062  261.39  55.47  38.40  46.94   1
2  1982   3.0  0.061  263.35  56.51  32.77  44.64   1
3  1982   4.0  0.058  264.99  52.86  28.74  40.80   1
4  1982   5.0  0.055  266.32  45.90  28.59  37.25   1
	 Дані після зміни індексів: 
    year  week    SMN     SMT    VCI    TCI    VHI  ID
0  1982   1.0  0.058  258.03  51.18  49.16  50.17  22
1  1982   2.0  0.062  261.39  55.47  38.40  46.94  22
2  1982   3.0  0.061  263.35  56.51  32.77  44.64  22
3  1982   4.0  0.058  264.99  52.86  28.74  40.80  22
4  1982   5.0  0.055  266.32  45.90  28.59  37.25  22


##### 5 Реалізувати процедури для формування вибірок наступного виду (включаючи елементи аналізу):
<ul>
    <li>Ряд VHI для області за рік, пошук екстремумів (min та max);</li>
    <li>Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка області;</li>
    <li>Аналогічно для помірних посух</li>
</ul>

 5.1 Ряд VHI для області за рік, пошук екстремумів (min та max)


In [111]:
def getvhi(df,id,year):
    vhi=df[(df['ID'] == id) & (df["year"] == str(year))][['year','week','VHI']]
    min=vhi['VHI'].min()
    max=vhi['VHI'].max()
    return vhi,min,max

vhi,min,max=getvhi(cdf,21,1984)
print(f"{vhi} \n {min=} \n {max=}")

     year  week    VHI
104  1984   1.0  32.74
105  1984   2.0  37.39
106  1984   3.0  41.66
107  1984   4.0  43.52
108  1984   5.0  44.30
109  1984   6.0  43.85
110  1984   7.0  44.11
111  1984   8.0  45.73
112  1984   9.0  46.84
113  1984  10.0  46.68
114  1984  11.0  45.26
115  1984  12.0  44.76
116  1984  13.0  42.44
117  1984  14.0  40.46
118  1984  15.0  37.95
119  1984  16.0  35.06
120  1984  17.0  33.92
121  1984  18.0  33.65
122  1984  19.0  34.51
123  1984  20.0  39.36
124  1984  21.0  43.47
125  1984  22.0  46.66
126  1984  23.0  50.01
127  1984  24.0  52.79
128  1984  25.0  56.03
129  1984  26.0  58.88
130  1984  27.0  61.03
131  1984  28.0  62.50
132  1984  29.0  64.27
133  1984  30.0  63.18
134  1984  31.0  59.62
135  1984  32.0  55.92
136  1984  33.0  54.04
137  1984  34.0  52.28
138  1984  35.0  50.61
139  1984  36.0  47.29
140  1984  37.0  43.00
141  1984  38.0  41.37
142  1984  39.0  40.96
143  1984  40.0  37.42
144  1984  41.0  36.37
145  1984  42.0  34.85
146  1984  

5.2 Ряд VHI за всі роки для області, виявити роки з екстремальними посухами, які торкнулися більше вказаного відсотка (області) VHI


Повернути VHI для області. + VHI <15 
+ VHI > якогось значення

Зміст VHI-індексу
<ul>
    <li>VHI < 40 – стресові умови; </li>
    <li> VHI > 60 – сприятливі умови;</li>
    <li> VHI < 15 – екстремальна посуха;</li>
    <li> VHI < 35 – помірна посуха.</li>
</ul>

In [145]:
def extreme_drought(df,vhivalue,provinceid):
    vhi_more_than_value=df[(df['VHI']> vhivalue) & (df['ID'] == provinceid)][['year','week','VHI']]
    vhi_drought=df[(df['VHI'] < 15) & (df['ID'] == provinceid)][['year','week','VHI']]
    
    return vhi_more_than_value,vhi_drought
v,d =extreme_drought(cdf,50,2)
print(f"VHI більше за введене значення: \n{v}")
print(f"Екстремальна посуха: \n{d}")


VHI більше за введене значення: 
      year  week    VHI
17    1982  18.0  52.62
18    1982  19.0  53.55
19    1982  20.0  53.81
20    1982  21.0  55.32
21    1982  22.0  55.26
...    ...   ...    ...
2161  2023  30.0  66.31
2162  2023  31.0  66.02
2163  2023  32.0  62.83
2164  2023  33.0  57.23
2165  2023  34.0  51.97

[753 rows x 3 columns]
Екстремальна посуха: 
     year  week    VHI
979  2000  44.0  12.99
980  2000  45.0  10.89
981  2000  46.0  11.46
982  2000  47.0  12.45
983  2000  48.0  14.71


5.3 Аналогічно для помірних посух

In [146]:
def moderate_drought(df,provinceid,vhivalue):
    vhi_more_than_value=df[(df['VHI']> vhivalue) & (df['ID'] == provinceid)][['year','week','VHI']]
    vhi_moderate=df[( df['VHI'] > 15) & ( df['VHI'] < 35) & (df['ID'] == provinceid)][['year','week','VHI']]
    return  vhi_more_than_value, vhi_moderate

v,m = moderate_drought(cdf,2,80)
print(f"VHI більше за введене значення: \n{v}")
print(f"\n Помірна посуха: \n{m}")

VHI більше за введене значення: 
      year  week    VHI
647   1994  24.0  80.74
648   1994  25.0  81.31
1008  2001  21.0  81.91
1009  2001  22.0  81.77

 Помірна посуха: 
      year  week    VHI
6     1982   7.0  34.68
7     1982   8.0  33.30
8     1982   9.0  32.61
9     1982  10.0  31.66
10    1982  11.0  31.18
...    ...   ...    ...
2124  2022  45.0  30.45
2125  2022  46.0  31.82
2174  2023  43.0  32.95
2175  2023  44.0  30.06
2176  2023  45.0  30.60

[225 rows x 3 columns]
